In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [2]:
matplotlib.rcParams['figure.figsize'] = [20, 10]

Vamos a aplicar regularización $l_2$  y regularization $l_1$ usando el siguiente ejemplo de una competición de  <kaggle.com> llamado __House Prices: Advanced Regression Techniques__, ver <https://www.kaggle.com/c/house-prices-advanced-regression-techniques>. 

Por favor baja el dataset `train.cvs`. 

In [4]:
path = "/home/ainhoa/Master/Datasets/house_train.csv"
houses = pd.read_csv(path)
houses.columns.values

array(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu',
       'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars',
       'GarageArea', 'GarageQual', 'GarageCond', 'Pav

### Seleccionamos para evitar complejidad las columnas numericas din elementos NAs

In [5]:
numeric_columns = houses.select_dtypes(include=[np.number]).columns
all_non_numeric_columns = houses.select_dtypes(exclude=[np.number]).columns

In [6]:
Xy = houses[numeric_columns]
target_col = "SalePrice"

Borrando las columnas con Nas values

In [7]:
cols_with_na = []
for col in Xy.columns.values:
    nas = sum(Xy[col].isna())
    if nas > 0:
        cols_with_na.append(col)
        print(col, sum(Xy[col].isna()))
        
Xy = Xy.drop(cols_with_na, axis=1)

LotFrontage 259
MasVnrArea 8
GarageYrBlt 81


Hacemos separación de datos de entrenamiento, test y validación:

In [8]:
X = Xy.drop(target_col, axis=1)
y = Xy[target_col]

X_train_dev, X_test, y_train_dev, y_test = train_test_split(X, y, random_state=666, test_size=0.2)
X_train, X_dev, y_train, y_dev = train_test_split(X_train_dev, y_train_dev, random_state=667, test_size=0.25)

## Método 1: Simple regresión lineal

In [9]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X_train, y_train)
y_dev_hat = reg.predict(X_dev)
print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)))

34461.6971629451


## Método 2: Regresiónn (l2-regularización)

Dado el modeo lineal: 


$$\hat{y} = a_1x_1 + \ldots + a_nx_n + b$$



Para regularización $l_2$, la función objetivo es :

$$l(\hat{Y}) = \frac{1}{N}\sum_i^N \left(y_i - \hat{y}_i\right)^2 + \alpha \sum(a_1^2 + \ldots + a_n^2)$$

Este es llamado Ridge.




In [10]:
from sklearn.linear_model import Ridge

for i in range(-10, 10):
    alpha = 2**i
    ridge_reg = Ridge(alpha=alpha)
    ridge_reg.fit(X_train, y_train)
    y_dev_hat = ridge_reg.predict(X_dev)
    print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)), alpha)

34461.678180100265 0.0009765625
34461.65919907181 0.001953125
34461.6212424607 0.00390625
34461.545351014065 0.0078125
34461.39365516013 0.015625
34461.09061110927 0.03125
34460.48590964141 0.0625
34459.28202148067 0.125
34456.89605317226 0.25
34452.209386845054 0.5
34443.16215814913 1
34426.2627763831 2
34396.510449337984 4
34348.95102038735 8
34282.42188337413 16
34202.482052117935 32
34125.16701255966 64
34099.00768052047 128
34245.40793649032 256
34744.59023643727 512


¿Cuál es entonces el mejor modelo?

## Lasso Regression (l1 regularización)

En este caso la función objetivo es:

$$l(\hat{Y}) = \frac{1}{N}\sum_i^N \left(y_i - \hat{y}_i\right)^2 + \alpha \sum(|a_1| + \ldots + |a_n|)$$

Este es llamado Lasso.

In [11]:
from sklearn.linear_model import Lasso

for i in range(-10, 10):
    alpha = 2**i
    lasso_reg = Lasso(alpha=alpha)
    lasso_reg.fit(X_train, y_train)
    y_dev_hat = lasso_reg.predict(X_dev)
    print(np.sqrt(mean_squared_error(y_dev, y_dev_hat)), alpha)

34461.69542173473 0.0009765625
34461.69368052823 0.001953125
34461.6901981263 0.00390625
34461.683233412696 0.0078125
34461.66930413723 0.015625
34461.64144640941 0.03125
34461.58573441035 0.0625
34461.47432458774 0.125
34461.2515619947 0.25
34460.80626696905 0.5
34459.91658585277 1
34458.14084437147 2
34454.60385885485 4
34447.587976514675 8
34433.78811793976 16
34407.119962130426 32
34357.519619665545 64
34288.09525971574 128
34229.02126168853 256
34270.67818181146 512


¿Cuál es el mejor modelo?


### Interpretación de los coeficientes en Lasso

En Lasso hay un interesante efecto en anular bastantes coeficientes
cuando aplicamos regularización. Veamos un ejemplo para $\alpha = 100000$

In [12]:
lasso_reg = Lasso(alpha=100000)
lasso_reg.fit(X_train, y_train)
lasso_reg.coef_

array([-3.17677896e+00, -5.19281675e+01,  1.81804546e-01,  0.00000000e+00,
        0.00000000e+00,  4.60597162e+02,  3.74462039e+02,  1.25190335e+01,
        0.00000000e+00, -0.00000000e+00,  2.42657167e+01,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  6.60160074e+01,  0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        6.00591708e+01,  3.89130770e+01, -0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  6.20233278e+01, -1.61332470e+02, -5.81768963e+00,
        0.00000000e+00, -0.00000000e+00])

# Elastic Net: ambas regularizaciones al mismo timepo

Hay un tercer tipo de regularización que trabaja muy bien que es la combinación de $l_2$ y $l_1$:


$$l(\hat{Y}) = \frac{1}{N}\sum_i^N \left(y_i - \hat{y}_i\right)^2 + 
\alpha_1 \sum(|a_1| + \ldots + |a_n|)+
\alpha_1 \sum((a_1)^2 + \ldots + (a_n)^2)
$$

El modelo lineal con esta función objetivo es a menudo llamdadoElastic Net. Esta tienes dos parámetros  `alpha` y `l1_ratio` y la relación entre ellos es la siguiente:

$$\alpha_1 = \texttt{alpha}\cdot\texttt{l1_ratio}$$

$$\alpha_2 = \texttt{alpha}\cdot(\texttt{1 - l1_ratio})$$

Veamos en el ejemplo anterior:

In [13]:
from sklearn.linear_model import ElasticNet
import warnings
warnings.filterwarnings("ignore")
alpha_ratio_score = []

for i in range(-10, 10):
    for j in range(20):
        alpha = 2**i
        l1_ratio = 0.6**j
        en_reg = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
        en_reg.fit(X_train, y_train)
        y_dev_hat = en_reg.predict(X_dev)
        alpha_ratio_score.append([alpha, l1_ratio, np.sqrt(mean_squared_error(y_dev, y_dev_hat)), alpha, l1_ratio])
        
scores = pd.DataFrame({
    "alpha": [ars[0] for ars in alpha_ratio_score],
    "ratio": [ars[1] for ars in alpha_ratio_score],
    "score": [ars[2] for ars in alpha_ratio_score]
})

In [36]:
scores.sort_values('score').head()

,alpha,ratio,score
147,0.125,0.027994,34049.422441
148,0.125,0.016796,34049.445167
146,0.125,0.046656,34049.459889
149,0.125,0.010078,34049.474806
150,0.125,0.006047,34049.498295


In [40]:
##from sklearn.preprocessing import StandardScaler, MinMaxScaler
##scaler = StandardScaler()
#scaler = MinMaxScaler()
##reg = LinearRegression()
##scaler.fit(X_train)

##X_train_scaled = scaler.transform(X_train)
##X_dev_scaled = scaler.transform(X_dev)